In [114]:
import os
os.path.abspath(os.getcwd())

'C:\\Users\\mikha\\Dropbox\\mikhael_misc\\Projects\\544 Replication - Bruckner and Ciccone - Rainfall and Voting'

# Summary:

This work follows Bruckner and Ciccone (2011), who found that the negative exogenous economic shocks which followed droughts in sub-Saharan Africa were solid contributors to / predictors of democratic revolutions. 

I attempt to establish a relationship between droughts (again serving as negative exogenous economic shocks) and political change, which might take any of the following forms:
* Increased votes for a specific pro-farmer party (will have to investigate whether democrats or republicans are viewed as pro-farmer.)
* Increased probability of an incumbent losing
* Increased votes for the Democratic party, who generally favor greater redistribution (see: Acemoglu & Robinson)

  
  
 newline

The equation of interest takes the form \begin{equation} Y_{i,t} = \alpha \, * Drought_{i,t} + \beta'X_{i,t} + \phi_{i,t} + \psi_{i,t} + u_{i,t} \end{equation}

NOTE: will need to include (and interact) a dummy term for presidential election or not that year.

where 
* $Y_{i,t} = $ our outcome of interest - either vote share for a particular party; probability of an incumbent losing; etc.
* $\alpha = $ the effect of drought on the outcome of interest
* $X_{i,t} = $ characteristics of the district at time *t*, such as income, farming as a share of GDP, average farm size, etc.
* $\phi_{i,t} = $ district fixed effects
* $\psi_{i,t} = $ time fixed effects
* state fixed effects?

include willingness to elect other party? years since change of power? avg years for incumbent to stay in power?

# Data Import and Cleaning

## Import and Setup

In [2]:
import pandas as pd
import numpy as np

In [14]:
pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_colwidth', 199)

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

## Market Value by District Data

In [69]:
# from 
# https://www.nass.usda.gov/Publications/AgCensus/2012/Online_Resources/Congressional_District_Rankings/ , and
# https://www.nass.usda.gov/Publications/AgCensus/2017/Online_Resources/Congressional_District_Rankings/
farm_census_2012 = pd.read_csv(r"C:\Users\mikha\OneDrive\Desktop\Dropbox\MIKHAEL NEW\mikhael school\Grad School\Master's\Term 2 Classes\544\Replication\Mkt Value of Agr Products Sold\2012.csv")
farm_census_2017 = pd.read_csv(r"C:\Users\mikha\OneDrive\Desktop\Dropbox\MIKHAEL NEW\mikhael school\Grad School\Master's\Term 2 Classes\544\Replication\Mkt Value of Agr Products Sold\2017.csv")

farm_census_2012['year'] = 2012
farm_census_2017['year'] = 2017

# merge 2012 and 2017
farm_census = pd.concat([farm_census_2012, farm_census_2017])

del farm_census_2012, farm_census_2017

### Some Text Cleaning

In [70]:
# upper-case
farm_census['state'] = [x.upper() for x in farm_census['state']]

#remove non-alphabet characters
farm_census['state'] = farm_census['state'].str.replace('\d+', '')
farm_census['state'] = farm_census['state'].str.replace('/', '')

In [71]:
# narrow down districts we can focus the analysis on
usable_districts = {state:set() for state in farm_census['state']}

In [72]:
for state_dist in list(zip(farm_census['state'], farm_census['district'])):
    if state_dist[1] != 'At Large':
        usable_districts[state_dist[0]].add(int(state_dist[1]))
    else:
        usable_districts[state_dist[0]].add(state_dist[1])

## Election Data (House of Rep's)

Include Senate data (same source)? Only catch is it isn't as specific to the agricultural districts. Could still be useful as a baseline.

In [109]:
# from https://electionlab.mit.edu/data 
elec_df = pd.read_csv(r"C:\Users\mikha\OneDrive\Desktop\Dropbox\MIKHAEL NEW\mikhael school\Grad School\Master's\Term 2 Classes\544\Replication\1976-2018 US House Election Data.csv", engine='python')

#drop useless columns (one unique value and/or irrelevant info)
elec_df.drop(['office', 'candidate', 'mode', 'version'], axis=1, inplace=True)

#drop if not Democratic or Republican candidate (MAYBE KEEP LIBERTARIANS?)
elec_df = elec_df[elec_df['party'].isin(['DEMOCRAT','REPUBLICAN', 'LIBERTARIAN'])]

### Drop election observations corresponding to *non-agricultural* districts

Note that we may need to undo this later to create some sort of baseline

In [110]:
# DROP IF NOT A BIG AGRICULTURAL DISTRICT (that we have economic/agricultural data for)

# NOTE: still need to sort out "at large" stuff

# this loop just appends 0 or 1 to the vector "usable_obs" if we have economic/agricultural data for that district
usable_obs = []
for i, row in elec_df.iterrows():
    if row['state'] in usable_districts and row['district'] in usable_districts[row['state']]:
        usable_obs.append(1)
    else:
        usable_obs.append(0)

# here we bring this vector into the df        
elec_df['usable'] = usable_obs

del usable_obs

# finally, drop all observations that don't correspond to "usable" districts. 
# This leaves us with 2974 election observations
elec_df = elec_df[elec_df['usable']==1]

# To Do Next!

Figure out who won each election, and by how much, and whether they're an incumbent.

Then, a ton of data munging to do on Drought data

In [4]:
#create a "winner" variable
#first, create a dict

voting_dict

#create a "vote share" variable




### Drought Data

In [112]:
# from https://droughtmonitor.unl.edu/Data/DataDownload/ComprehensiveStatistics.aspx
drought_df = pd.read_csv(r"C:\Users\mikha\OneDrive\Desktop\Dropbox\MIKHAEL NEW\mikhael school\Grad School\Master's\Term 2 Classes\544\Replication\Drought Data.csv")

In [113]:
drought_df

,MapDate,FIPS,County,State,None,D0,D1,D2,D3,D4,ValidStart,ValidEnd,StatisticFormatID
0,20210209,2013,Aleutians East Borough,AK,100.00,0.00,0.0,0.0,0.0,0.0,2021-02-09,2021-02-15,1
1,20210202,2013,Aleutians East Borough,AK,100.00,0.00,0.0,0.0,0.0,0.0,2021-02-02,2021-02-08,1
2,20210126,2013,Aleutians East Borough,AK,100.00,0.00,0.0,0.0,0.0,0.0,2021-01-26,2021-02-01,1
3,20210119,2013,Aleutians East Borough,AK,100.00,0.00,0.0,0.0,0.0,0.0,2021-01-19,2021-01-25,1
4,20210112,2013,Aleutians East Borough,AK,100.00,0.00,0.0,0.0,0.0,0.0,2021-01-12,2021-01-18,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3547333,20000201,56045,Weston County,WY,100.00,0.00,0.0,0.0,0.0,0.0,2000-02-01,2000-02-07,1
3547334,20000125,56045,Weston County,WY,3.93,96.07,0.0,0.0,0.0,0.0,2000-01-25,2000-01-31,1
3547335,20000118,56045,Weston County,WY,100.00,0.00,0.0,0.0,0.0,0.0,2000-01-18,2000-01-24,1
3547336,20000111,56045,Weston County,WY,100.00,0.00,0.0,0.0,0.0,0.0,2000-01-11,2000-01-17,1
